# Lab: Bring your own custom container with Amazon SageMaker

## Overview
Here, we’ll show how to package a simple Python example which showcases the decision tree algorithm from the widely used scikit-learn machine learning package. The example is purposefully fairly trivial since the point is to show the surrounding structure that you’ll want to add to your own code so you can train and host it in Amazon SageMaker.

The ideas shown here will work in any language or environment. You’ll need to choose the right tools for your environment to serve HTTP requests for inference, but good HTTP environments are available in every language these days.

In this example, we use a single image to support training and hosting. This is easy because it means that we only need to manage one image and we can set it up to do everything. Sometimes you’ll want separate images for training and hosting because they have different requirements. Just separate the parts discussed below into separate Dockerfiles and build two images. Choosing whether to have a single image or two images is really a matter of which is more convenient for you to develop and manage.

If you’re only using Amazon SageMaker for training or hosting, but not both, there is no need to build the unused functionality into your container.



## Building the container
Docker provides a simple way to package arbitrary code into an image that is totally self-contained. Once you have an image, you can use Docker to run a container based on that image. Running a container is just like running a program on the machine except that the container creates a fully self-contained environment for the program to run. Containers are isolated from each other and from the host environment, so the way you set up your program is the way it runs, no matter where you run it.

Amazon SageMaker uses Docker to allow users to train and deploy arbitrary algorithms.

### Parts of the container
In the container directory are all the components you need to package the sample algorithm for Amazon SageMager:

```
.
|-- Dockerfile
|-- build_and_push.sh
`-- decision_trees
    |-- nginx.conf
    |-- predictor.py
    |-- serve
    |-- train
    `-- wsgi.py
```

Let’s discuss each of these in turn:

- Dockerfile describes how to build your Docker container image. More details below:
- build_and_push.sh is a script that uses the Dockerfile to build your container images and then pushes it to ECR. We’ll invoke the commands directly later in this notebook, but you can just copy and run the script for your own algorithms.
- decision_trees is the directory which contains the files that will be installed in the container.
- local_test is a directory that shows how to test your new container on any computer that can run Docker, including an Amazon SageMaker notebook instance. Using this method, you can quickly iterate using small datasets to eliminate any structural bugs before you use the container with Amazon SageMaker. We’ll walk through local testing later in this notebook.

In this simple application, we only install five files in the container.

The files that we’ll put in the container are:

- nginx.conf is the configuration file for the nginx front-end. Generally, you should be able to take this file as-is保持原样.
- predictor.py is the program that actually implements the Flask web server and the decision tree predictions for this app. You’ll want to customize the actual prediction parts to your application. Since this algorithm is simple, we do all the processing here in this file, but you may choose to have separate files for implementing your custom logic.
- serve is the program started when the container is started for hosting. It simply launches the gunicorn server(Gunicorn“ Green Unicorn”是Python Web服务器网关接口HTTP服务器。它是从Ruby的Unicorn项目移植而来的pre-fork worker模型。Gunicorn服务器与许多Web框架广泛兼容，这些Web框架实现简单，占用服务器资源少且速度相当快。) which runs multiple instances of the Flask app defined in predictor.py. You should be able to take this file as-is.
- train is the program that is invoked when the container is run for training. You will modify this program to implement your training algorithm.
- wsgi.py is a small wrapper used to invoke the Flask app. You should be able to take this file as-is. Web服务器网关接口 (Web Server Gateway Interface)是为Python语言定义的Web服务器和Web应用程序或框架之间的一种简单而通用的接口。

In summary, the two files you will probably want to change for your application are train and predictor.py

### The Dockerfile
The Dockerfile describes the image that we want to build. You can think of it as describing the complete operating system installation of the system that you want to run. A Docker container running is quite a bit lighter than a full operating system, because it takes advantage of Linux on the host machine for the basic operations.

For the Python science stack, we will start from a standard Ubuntu installation and run the normal tools to install the things needed by scikit-learn. Finally, we add the code that implements our specific algorithm to the container and set up the right environment to run under.

In [1]:
# cell 00
!pip install --upgrade aiobotocore

     |████████████████████████████████| 58 kB 774 kB/s  eta 0:00:01
     |████████████████████████████████| 8.4 MB 43.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 57.7 MB/s eta 0:00:01
     |████████████████████████████████| 270 kB 61.1 MB/s eta 0:00:01
     |████████████████████████████████| 159 kB 67.9 MB/s eta 0:00:01
     |████████████████████████████████| 191 kB 66.9 MB/s eta 0:00:01
  Created wheel for aiobotocore: filename=aiobotocore-2.1.2-py3-none-any.whl size=55993 sha256=f9ae1542a43b5abf611b2470607810cc3a5e755db4665e8dfd7e93a7073b9dec
  Stored in directory: /root/.cache/pip/wheels/a3/89/13/9c60f1de355c526583bb1c5712186fddd8f98f31ca1d78dc19
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-py3-none-any.whl size=3178 sha256=efe2daa3f7a9ecd4f8a66167294ea96051531422851b1233a4226ca21e57f3f0
  Stored in directory: /root/.cache/pip/wheels/6a/f5/9c/f8331a854f7a8739cf0e74c13854e4dd7b1af11b04fe1dde13
Successfully built aiobotocore idna-ssl
  Attempting uninsta

In [23]:
import zipfile

file_dir = 'scikit_bring_your_own.zip'
zip_File = zipfile.ZipFile(file_dir,'r')
zip_list = zip_File.namelist()

#打印详细信息
# zzz = zip_File.printdir()

#打印列表信息
# print(zip_list)

#解压
zip_File.extractall(r'.')

In [24]:
# cell 01

# !unzip scikit_bring_your_own.zip
!mv scikit_bring_your_own/data/ ./lab03_data/
!mv scikit_bring_your_own/container/ ./lab03_container/
!rm -rf scikit_bring_your_own
!cat lab03_container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 2 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM public.ecr.aws/lts/ubuntu:18.04

MAINTAINER Amazon AI <sage-learner@amazon.com>


RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python \
         nginx \
         ca-certificates \
         build-essential \
         file \
         python-dev \
    && rm -rf /var/lib/apt/lists/*

# Here we get all python packages.
# There's substantial overlap between scipy and numpy that we eliminate by
# linking them together. Likewise, pip leaves the install caches populated which uses
# a significant amount of space. These optimizations save a fair amount of space in the
# image, which reduces start up time.
RUN wget https://bootstrap.pypa.io/pip/2.7/get-pip.py && python get-pip.py && \
    pip install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gevent 

## Building and registering the container

In [25]:
# cell 02
!pip install sagemaker-studio-image-build

     |████████████████████████████████| 7.7 MB 17.5 MB/s eta 0:00:01
  Created wheel for sagemaker-studio-image-build: filename=sagemaker_studio_image_build-0.6.0-py3-none-any.whl size=13468 sha256=cb7ac2307f402c447abe9f0e1ded6774b83b5e59864e83356ba0d8ea44c973d7
  Stored in directory: /root/.cache/pip/wheels/aa/ba/c7/3106a98300b3a1e806a6d4d1438a23a6eb5a265c4a67ae273f
Successfully built sagemaker-studio-image-build
  Attempting uninstall: botocore
    Found existing installation: botocore 1.23.24
    Uninstalling botocore-1.23.24:
      Successfully uninstalled botocore-1.23.24
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.19.23 requires botocore==1.20.23, but you have botocore 1.20.112 which is incompatible.
aiobotocore 2.1.2 requires botocore<1.23.25,>=1.23.24, but you have botocore 1.20.112 which is incompatible.
You should consider upgrading via t

> *In the next cell, if you run into IAM permission issue related to CodeBuild, make sure that you follow the steps outlined in the instructions*

In [30]:
!pwd

/root/amazon-sagemaker-immersion-day


In [32]:
# %%sh
# cell 03


!cd lab03_container

!chmod +x lab03_container/decision_trees/train
!chmod +x lab03_container/decision_trees/serve

!sm-docker build ./lab03_container  --repository sagemaker-decision-trees:latest

Created ECR repository sagemaker-decision-trees
...[Container] 2022/03/16 14:18:22 Waiting for agent ping

[Container] 2022/03/16 14:18:23 Waiting for DOWNLOAD_SOURCE
[Container] 2022/03/16 14:18:26 Phase is DOWNLOAD_SOURCE
[Container] 2022/03/16 14:18:26 CODEBUILD_SRC_DIR=/codebuild/output/src530662399/src
[Container] 2022/03/16 14:18:26 YAML location is /codebuild/output/src530662399/src/buildspec.yml
[Container] 2022/03/16 14:18:26 Setting HTTP client timeout to higher timeout for S3 source
[Container] 2022/03/16 14:18:26 Processing environment variables
[Container] 2022/03/16 14:18:26 No runtime version selected in buildspec.
[Container] 2022/03/16 14:18:26 Moving to directory /codebuild/output/src530662399/src
[Container] 2022/03/16 14:18:26 Configuring ssm agent with target id: codebuild:d419fa0a-d877-4f16-a3ec-8cb8ed70f2fc
[Container] 2022/03/16 14:18:26 Successfully updated ssm agent configuration
[Container] 2022/03/16 14:18:26 Registering with agent
[Container] 2022/03/16 14:

## Using the container
Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [ ]:
# cell 04

# S3 prefix
prefix = 'DEMO-scikit-byo-iris'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

The session remembers our connection parameters to SageMaker. We’ll use it to perform all of our SageMaker operations.

In [ ]:
# cell 05

import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

When training large models with huge amounts of data, you’ll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we’re using some the classic Iris dataset, which we have included.

We can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket.

In [ ]:
# cell 06

WORK_DIRECTORY = 'lab03_data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In order to use SageMaker to fit our algorithm, we’ll create an Estimator that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

- The container name. This is constructed as in the shell commands above.
- The role. As defined above.
- The instance count which is the number of machines to use for training.
- The instance type which is the type of machine to use for training.
- The output path determines where the model artifact will be written.
- The session is the SageMaker session object that we defined above.

Then we use `fit()` on the estimator to train against the data that we uploaded above.

In [ ]:
# cell 07

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-decision-trees:latest'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, instance_count=1, instance_type='ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)
file_location = data_location + '/iris.csv'
tree.fit(file_location)

## Hosting your model
You can use a trained model to get real time predictions using HTTP endpoint. Follow these steps to walk you through the process.

Deploying the model to SageMaker hosting just requires a deploy call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [ ]:
# cell 08
from sagemaker.serializers import CSVSerializer
predictor = tree.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge', serializer=CSVSerializer())

## Preparing test data to run inferences

In order to do some predictions, we’ll extract some of the data we used for training and do predictions against it. This is, of course, bad statistical practice, but a good way to see how the mechanism works.

In [ ]:
# cell 09

shape=pd.read_csv(file_location, header=None)
shape.sample(3)

In [ ]:
# cell 10

# drop the label column in the training set
shape.drop(shape.columns[[0]],axis=1,inplace=True)
shape.sample(3)

In [ ]:
# cell 11

import itertools

a = [50*i for i in range(3)]
b = [40+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data=shape.iloc[indices[:-1]]

## Run predictions

Prediction is as easy as calling predict with the predictor we got back from deploy and the data we want to do predictions with. The serializers take care of doing the data conversions for us.

In [ ]:
# cell 12

print(predictor.predict(test_data.values).decode('utf-8'))

## Cleanup
After completing the lab, use these steps to [delete the endpoint](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html) or run the following code


In [ ]:
# cell 13
sess.delete_endpoint(predictor.endpoint_name)

In [ ]:
# cell 14
!rm -rf lab03_container lab03_data